### Imports

In [5]:
import pickle
import torch
import torch.nn as nn
import numpy as np

import scripts.data_loader as data_loader
import scripts.model.training as training
import scripts.model.evaluation as evaluation

from dataclasses import dataclass, asdict
from scripts.model.types import * 
from models.model1 import OB_05Model

# Training

In [6]:
training_dataset, validation_dataset, testing_dataset = data_loader.split_images_dataset()
torch.save(training_dataset, "models/training_dataset.pth")
torch.save(validation_dataset, "models/validation_dataset.pth")
torch.save(testing_dataset, "models/testing_dataset.pth")

training_set_loader = data_loader.create_data_loader(training_dataset)
validation_set_loader = data_loader.create_data_loader(validation_dataset)
testing_set_loader = data_loader.create_data_loader(testing_dataset)

In [7]:
learning_rate = 0.001
model = OB_05Model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

training_config = training.TrainingConfig(
    training_set_loader=training_set_loader,
    validation_set_loader=validation_set_loader,
    testing_set_loader=testing_set_loader,

    epochs=10,
    learning_rate=learning_rate,

    classes=data_loader.get_trainset().classes,
    model=model,
    criterion=criterion,
    optimizer=optimizer
)

In [8]:
training_logger = training.train_model(training_config)

with open("models/training_logger.pkl", "wb") as file:
    pickle.dump(training_logger, file)

torch.save(model.state_dict(), 'models/model.pth')

Epoch 1/10:
	Training precision: 0.4407
	Training recall: 0.4402
	Training accuracy: 0.7259
	Training f1-score: 0.4346

	Validation precision: 0.7272
	Validation recall: 0.6809
	Validation accuracy: 0.8415
	Validation f1-score: 0.6635


Epoch 2/10:
	Training precision: 0.6130
	Training recall: 0.6132
	Training accuracy: 0.8114
	Training f1-score: 0.6121

	Validation precision: 0.8048
	Validation recall: 0.7383
	Validation accuracy: 0.8761
	Validation f1-score: 0.7102


Epoch 3/10:
	Training precision: 0.6767
	Training recall: 0.6744
	Training accuracy: 0.8435
	Training f1-score: 0.6725

	Validation precision: 0.7912
	Validation recall: 0.7707
	Validation accuracy: 0.8905
	Validation f1-score: 0.7649


Epoch 4/10:
	Training precision: 0.6898
	Training recall: 0.6874
	Training accuracy: 0.8497
	Training f1-score: 0.6859

	Validation precision: 0.7631
	Validation recall: 0.7630
	Validation accuracy: 0.8862
	Validation f1-score: 0.7627


Epoch 5/10:
	Training precision: 0.7232
	Training re

# Evaluation/Testing

In [10]:
with open("models/training_logger.pkl", "rb") as file:
    training_logger = pickle.load(file)

print(training_logger)

TrainingLogger(training_confusion_matrix_history=[array([[257, 101,  51,  28],
       [177,  84,  47,  35],
       [ 50,  16, 227, 138],
       [ 58,  23, 164, 164]]), array([[289, 127,   5,  16],
       [169, 145,  12,  17],
       [  3,  12, 298, 118],
       [ 10,  12, 110, 277]]), array([[320, 105,   5,   7],
       [168, 148,  13,  14],
       [  2,   5, 346,  78],
       [  9,   6,  95, 299]]), array([[323,  99,   6,   9],
       [163, 154,   9,  17],
       [  4,  10, 339,  78],
       [  8,   5,  79, 317]]), array([[318, 110,   1,   8],
       [144, 179,  10,  10],
       [  3,   4, 349,  75],
       [  5,   9,  56, 339]]), array([[332,  92,   4,   9],
       [132, 189,  12,  10],
       [  1,   4, 369,  57],
       [  7,   2,  43, 357]]), array([[329,  95,   4,   9],
       [125, 197,   9,  12],
       [  3,   2, 384,  42],
       [  2,  11,  43, 353]]), array([[341,  85,   2,   9],
       [ 95, 229,   7,  12],
       [  3,   4, 386,  38],
       [  8,   6,  42, 353]]), array(

In [ ]:
loaded_testing_dataset = torch.load('models/testing_dataset.pth')
testing_set_loader = data_loader.create_data_loader(loaded_testing_dataset)

print(next(iter(loaded_testing_dataset)))

In [ ]:
model = OB_05Model()
model.load_state_dict(torch.load('models/model.pth'))
evaluation_result = evaluation.evaluate_model(model, testing_set_loader)

In [ ]:
true_positives, false_positives, true_negatives, false_negatives = evaluation_result.calculate_metrics()

print(f'Total:\t\t\t\t{evaluation_result.total_tests()} tests\n'
      f'True positives:\t\t{np.sum(true_positives)}\n'
      f'False positives:\t{np.sum(false_positives)}\n'
      f'True negatives:\t\t{np.sum(true_negatives)}\n'
      f'False negatives:\t{np.sum(false_negatives)}\n'
      f'Precision:\t\t\t{evaluation_result.calculate_precision():.4f}\n'
      f'Recall:\t\t\t\t{evaluation_result.calculate_recall():.4f}\n'
      f'Accuracy:\t\t\t{evaluation_result.calculate_accuracy():.4f}\n'
      f'F1-Score:\t\t\t{evaluation_result.calculate_f1_score():.4f}\n')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

confusion_matrix = evaluation_result.confusion_matrix

hyperparameters = {
    'Learning Rate': 0.001,
    'Batch Size': 32,
    'Epochs': 10,
    'Optimizer': 'Adam'
}

# Function to plot confusion matrix
def plot_confusion_matrix(confusion_matrix, hyperparameters):
    plt.figure(figsize=(10, 8))
    
    # Plot the confusion matrix
    ax = plt.subplot(121)
    sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    
    # Plot the hyperparameters
    ax = plt.subplot(122)
    ax.axis('off')
    text = '\n'.join([f'{key}: {value}' for key, value in hyperparameters.items()])
    plt.text(0.5, 0.5, text, ha='center', va='center', fontsize=12)
    plt.title('Training Hyperparameters')
    
    plt.tight_layout()
    plt.show()

# Example usage
plot_confusion_matrix(confusion_matrix, hyperparameters)

TypeError: unsupported operand type(s) for |: 'list' and 'function'